In [1]:
!pip install 'bs4==0.0.1' 'lxml==4.9.2' 'googlemaps==4.10.0' 'python-dotenv'

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 10.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=35c84b212acf8843970cba83395589bc2d5a2d3ba700f4f233128a8449597cde
  Stored in directory: /Users/Diogo/Library/Caches/pip/wheels/d4/c8/5b/b5be9c20e5e4503d04a6eac8a3cd5c2393505c29f02bea0960
  Created wheel for lxml: filename=lxml-4.9.2-cp311-cp311-macosx_14_0_arm64.whl size=1522027 sha256=3fd59452946682079a113eef7e23bfb692e8cc2b0d0a378c1508fcf113cf996e
  Stored in directory: /Users/Diogo/Library/Caches/pip/wheels/fb/5b/f7/0a27880b4a007daeff53a196d01901627f640392b7e76e76e5
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40711 sha256=45011ae54c2d06292591ff668ba11db8b91f971dfdb92b3c18d366e7219f8ea3
  Stored in directory: /Users/Diogo/Library/Caches/pip/wheels/f1/09

In [19]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import unquote
from dotenv import dotenv_values
import json
import time
import googlemaps

env = dotenv_values()

# API key for places, directions, geocode and another to maps js
gmaps = googlemaps.Client(key=env["GOOGLE_KEY"])

locations = []
geo = []
names = []
url = []
food = []
food_desc = []
contact = []
work_hours = []

def get_google(name, query, retry=3):
    try:
        result = gmaps.geocode(unquote(query))
        if len(result) > 1:
            place = gmaps.places(unquote(name))["results"]
            if len(place) == 1:
                result = place
        if not result: raise Exception()
        return result
    except Exception as e:
        if retry:
            time.sleep(1)
            return get_google(name, query, retry-1)
        else:
            print(f"query error {query}")
            print(e)
            return False

def request(url):
    r = Request(url, headers={'User-Agent' : "Magic Browser"})
    resp = urlopen(r)
    return BeautifulSoup(resp, "lxml")

for i in range(1, 94):
    soup = request(f"https://comidadibuteco.com.br/butecos/page/{i}/")
    inner = soup.findAll('div', {"class": "item"})
    for div in inner:
        name = div.find('h2').text
        links = div.findAll('a')
        if name and (len(links) == 2):
            u = links[0].get('href')
            s = request(u)
            data = s.find('div', {"class": "section-text"})
            addr = ''
            snack_details = ''
            phone = ''
            work = ''
            try:
                place = data.findAll('p')
                next_p = ''
                sanity_data = data.text.replace('\n', '').replace('\t', '').replace('\r', '')
                try:
                    snack = data.findAll('strong')[0].text.replace('\n', '').replace('\t', '').replace('\r', '')
                    addr_index = sanity_data.index('Endereço')
                except Exception as e:
                    raise e
                snack_details = sanity_data[len(snack):addr_index].split(':')[-1]

                if 'Telefone' in sanity_data:
                    phone_index = sanity_data.index('Telefone')
                    addr = sanity_data[addr_index+len('Endereço'): phone_index].split(':')[-1]
                    if 'Horário' in sanity_data:
                        work_index = sanity_data.index('Horário')
                        phone = sanity_data[phone_index+len('Telefone'): work_index].split(':')[-1]
                        work = sanity_data[work_index+len('Horário'):].split(':')[-1]
                    else:
                        phone = sanity_data[phone_index+len('Telefone'):].split(':')[-1]
                elif 'Horário' in sanity_data:
                    work_index = sanity_data.index('Horário')
                    addr = sanity_data[addr_index+len('Endereço'): work_index].split(':')[-1]
                    work = sanity_data[work_index+len('Horário'):].split(':')[-1]
                else:
                    addr = sanity_data[addr_index+len('Endereço'):].split(':')[-1]
                    
            except:
                print(f'fallback to maps link on {name}')
                addr = links[1].get('href').split('query=')[-1]
                
            if not addr:
                print(f"no addr found for {name}")
                continue
            geocode_result = get_google(name, addr.replace('|', '-'))
            if geocode_result:
                geo.append(geocode_result[0]["geometry"]["location"])
            else:
                print(f"no place found for {name}")
                geo.append({})
            url.append(u)
            names.append(name.strip())
            food.append(snack.strip())
            food_desc.append(snack_details.strip())
            contact.append(phone.strip())
            work_hours.append(work.strip().replace('hQ', 'h | Q').replace('hT', 'h | T').replace('hS', 'h | S').replace('hD', 'h | D'))
        else:
            continue
    print(f"foi página {i}")
    time.sleep(10)
for i in range(len(names)):
    locations.append({
        "name": names[i],
        "geo": geo[i],
        "url": url[i],
        "food": food[i],
        "food_desc": food_desc[i],
        "contact": contact[i],
        "work_hours": work_hours[i]
    })

fallback to maps link on A Porcaria
foi página 1
foi página 2
foi página 3
foi página 4
fallback to maps link on Assada e Cia
foi página 5
foi página 6
fallback to maps link on Bar Brasil
foi página 7
foi página 8
fallback to maps link on Bar da Praça
foi página 9
foi página 10
foi página 11
foi página 12
no addr found for Bar do Jordão
foi página 13
foi página 14
foi página 15
foi página 16
foi página 17
foi página 18
foi página 19
foi página 20
fallback to maps link on Bar Rigudos
foi página 21
fallback to maps link on Barile Bar
foi página 22
foi página 23
fallback to maps link on Berlim Delícias Pra Você
fallback to maps link on Biergarten Manaus
foi página 24
foi página 25
foi página 26
foi página 27
foi página 28
fallback to maps link on Boteco do Ferrari
fallback to maps link on Boteco do Libanês
no addr found for Boteco do Mala
foi página 29
foi página 30
foi página 31
foi página 32
foi página 33
foi página 34
foi página 35
foi página 36
foi página 37
fallback to maps link on C

In [21]:
locations

[{'name': '14 Bis Empório da Cerveja',
  'geo': {'lat': -18.9264547, 'lng': -48.2763163},
  'url': 'https://comidadibuteco.com.br/buteco/14-bis-emporio-da-cerveja/',
  'food': 'Meia Lua de Camarão',
  'food_desc': 'Risole de Camarão com sabor inesquecível.',
  'contact': '(34) 3219-3971',
  'work_hours': 'Terças, das 10h às 00hQuartas, das 10h às 00hQuintas, das 10h às 00hSextas, das 10h às 00hSábados, das 10h às 00hDomingos, das 10h às 16h'},
 {'name': '1960 Bar',
  'geo': {'lat': -16.6810272, 'lng': -49.2825398},
  'url': 'https://comidadibuteco.com.br/buteco/1960-bar/',
  'food': 'Croqueta de Cora',
  'food_desc': 'Croqueta de vaca atolada ao molho Coralina e gremolata.',
  'contact': '',
  'work_hours': 'Segunda de 17h à 22h | Terça de 17h à 22h | Quinta de 17h à 22h | Quarta de 17h à 22h | Sexta de 17h à 22h | Sabado de 11h à 16h |'},
 {'name': '222',
  'geo': {'lat': -19.9456419, 'lng': -43.9297118},
  'url': 'https://comidadibuteco.com.br/buteco/222-2/',
  'food': 'Uai? Carajé',

In [22]:
with open("places.json", "w") as f:
    f.write(json.dumps(locations, ensure_ascii=False))

In [23]:
len(names)

1105

In [24]:
len(geo)

1105

In [25]:
len(url)

1105

In [26]:
len(food)

1105

In [27]:
len(food_desc)

1105

In [28]:
len(list(filter(lambda item: bool(item), geo)))

1105